In [1]:
config = {}
config['data'] = {"qm9_broad_ir_path":'/home2/kanakala.ganesh/ir_data/qm9_broad_ir.pkl',
                  "vocab_path":'/home2/kanakala.ganesh/CLIP_PART_1/data/qm9_vocab.pkl',
                  "datafiles" : {
                        'train': '/home2/kanakala.ganesh/ir_data/raw_train.pickle',
                        'test':  '/home2/kanakala.ganesh/ir_data/raw_test.pickle',
                        'val':   '/home2/kanakala.ganesh/ir_data/raw_val.pickle'
                        },
                  "normalization" : "unit",
                  "shuffle": True,
                  "batch_size":128,
                  "seq_len":70,
                  "splits":[0.9, 0.1, 0.1],
                  "num_workers":16
                }

In [2]:
from PrepareData import prepare_data
dataloaders, max_charge, num_species = prepare_data(config)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [3]:

for i , data in enumerate(dataloaders['train']):
    data
    for j in data:
        print(j, data[j].shape)
    break

index torch.Size([128])
decoder_inp torch.Size([128, 70])
decoder_tgt torch.Size([128, 70])
IR torch.Size([128, 1801])
tgt_padding_mask torch.Size([128, 70])
num_atoms torch.Size([128])
charges torch.Size([128, 29])
positions torch.Size([128, 29, 3])
one_hot torch.Size([128, 29, 5])
atom_mask torch.Size([128, 29])
edge_mask torch.Size([107648, 1])


In [5]:
config['spectra_encoder'] = {
    'd_ff': 1024,
    'dropout': 0.0,
    'dropout_emb': 0.1,
    'h_dim': 128,
    'max_time_steps': 1000,
    'num_heads': 5,
    'num_layers': 5,
    'output_size': 512,
    'patch_size': 7,
    'use_clf_token': True,
}

In [6]:
from models.vit import ViT

model = ViT(
            patch_size = config['spectra_encoder']['patch_size'], 
            num_layers = config['spectra_encoder']['num_layers'], 
            h_dim = config['spectra_encoder']['h_dim'], 
            num_heads =config['spectra_encoder']['num_heads'], 
            output_size = config['spectra_encoder']['output_size'], 
            d_ff=config['spectra_encoder']['d_ff'], 
            max_time_steps=config['spectra_encoder']['max_time_steps'], 
            use_clf_token=config['spectra_encoder']['use_clf_token'],
            dropout = config['spectra_encoder']['dropout'],
            dropout_emb = config['spectra_encoder']['dropout_emb']
            
        )

In [14]:
import torch
def forward_pass(model, dataloader):
    for i , data in enumerate(dataloader):
        data
        break
    
    spectra = data['IR']
    spectra = torch.unsqueeze(spectra,1)
    spectra = torch.unsqueeze(spectra,1)
    spectra = spectra.to(torch.float32)
    # [B, C, H, W]
    preds = model(spectra)
    
    
    print(preds.shape)
    return preds

In [15]:
out = forward_pass(model, dataloaders['train'])

torch.Size([128, 512])
